In [271]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import csv
import random
import time
import seaborn as sns 
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import colorsys
%matplotlib qt5
class Bra:
    def __init__(self, w,rg_w,device,x_train,y_train,x_test,y_test,eps,error_limited,nmax,alpha,momentum):
        self.__w = [torch.tensor(item, dtype=torch.float32, requires_grad=rg_w[rq]) for rq, item in enumerate(w)]  
        self.__rg_w=rg_w
        #definir la puce utilise pour l'acceleration du calcul de tenseur choix: 'cuda'-nvidia gpu / 'mps'-apple gpu  /'cpu'-le cpu de pc
        self.__device=device
        #initialisation des parametres
        self.__index=random.choice([i for i in range(len(x_train))])
        #point aleatoire depart
        self.__xs=[torch.tensor(item, dtype=torch.float32, requires_grad=False) for item in x_train.T.iloc[:,self.__index]] 
        self.__ys=torch.tensor(y_train[self.__index], dtype=torch.float32, requires_grad=False)
        #list des donnes de nuage de points
        self.__x_train=x_train
        self.__y_train=y_train
        self.__x_test=x_test
        self.__y_test=y_test
        self.__eps=eps
        self.__nmax=nmax
        #Taux d'apprentissage
        self.__alpha=alpha
        self.__momentum=momentum
        self.__n=0
        self.__dx=1.0000
        self.__error=1
        self.__error_limited=error_limited
        self.__condition=True
        self.__time_initial=time.time()
        self.__draw=[[] for i in range(len(self.__w)+5)]
        
    # getters
    @property
    def w(self):
        return self.__w  
    @property
    def device(self):
        return self.__device
    @property
    def n(self):
        return self.__n
    @property
    def dx(self):
        return self.__dx
    @property
    def error(self):
        return self.__error
    @property
    def index(self):
        return self.__index
    @property
    def alpha(self):
        return self.__alpha
    @property
    def draw(self):
        return self.__draw
    @property
    def xs(self):
        return self.__xs
    @property
    def ys(self):
        return self.__ys
    @property
    def draw(self):
        return self.__draw
    # setters
    @w.setter
    def w(self, w):
        w=[torch.tensor(item, dtype=torch.float32, requires_grad=self.__rg_w[rq]) for rq, item in enumerate(w)]
        self.__w = w
    @n.setter
    def n(self,n):
        self.__n=n 
    @dx.setter
    def dx(self,values):
        w = values
        dx=0
        for i in range(len(self.__w)):
            dx+=(self.__w[i].item()-w[i])**2
        self.__dx=np.sqrt(dx) 
    @error.setter
    def error(self,error):
        self.__error=error
    @index.setter
    def index(self,index):
        self.__index=index
    @alpha.setter
    def alpha(self,alpha):
        self.__alpha=alpha
    @xs.setter
    def xs(self,xs):
        self.__xs=xs
    @ys.setter
    def ys(self,ys):
        self.__ys=ys 
    @draw.setter
    def draw(self,element):
        for i in range(len(element)):
            self.__draw[i].append(element[i])

    def append_list(self,list1,list2):
        for i in range(len(list2)):
            list1.append(list2[i])
            
    #fonction du cout a minimiser    
    def j(self, w):
        dot_product = torch.dot(torch.stack(w), torch.stack(self.__xs))
        return abs(self.__ys - torch.exp(dot_product) / (1 + torch.exp(dot_product)))
    def get_error(self):
        error=0
        for i in range(len(self.__x_train)):
            dot_product = torch.dot(torch.stack(self.__w), torch.stack([torch.tensor(value) for value in self.__x_train.T.iloc[:,i]]))

            error+=abs(self.__y_train[i] - torch.exp(dot_product) / (1 + torch.exp(dot_product)))
        return error.item()/len(self.__x_train)
    def classification(self,W):
        w=[element.item() for element in W]
        n=0
        for i in range(len(self.__x_test)):
            dot_product = np.dot(w, self.__x_test.T.iloc[:,i])
            n_class=round(np.exp(dot_product) / (1 + np.exp(dot_product)))
            if n_class== self.__y_test[i]:
                n+=1
        #print("n_iteration->",self.__n,"------>test accuracy {} %".format(n/len(self.__x_test)*100))
        return n/len(self.__x_test)             
    #fonction pour calculer les gradient  
    def getgradient(self):
        grad_w = np.zeros((len(self.__w)+1),dtype=np.float32)
        torch.device(self.__device)
        w=self.__w
        result = self.j(w)
        result.backward()
        for i in range(len(self.__w)):
            grad_w[i] = w[i].grad.item() if w[i].grad is not None else 0.0
        return grad_w
    def generate_distinct_colors(self,num_colors):
        colors = []
        saturation = 0.7  
        value = 0.9  

        for i in range(num_colors):
            hue = (i / num_colors) % 1.0  
            rgb = colorsys.hsv_to_rgb(hue, saturation, value)
            color_hex = '#{:02x}{:02x}{:02x}'.format(
                int(rgb[0] * 255),
                int(rgb[1] * 255),
                int(rgb[2] * 255)
            )
            colors.append(color_hex)
        return colors

    def Plot_curves(self):
        colors = self.generate_distinct_colors(len(self.__w))
        fig1, ax1 = plt.subplots()
        ax1.plot(self.__draw[0],self.__draw[-3],label='global_error')
        ax1.set_title('global_error')
        ax1.set_xlabel('n_iteration')
        ax1.legend(loc='upper right')

        fig2, ax2 = plt.subplots()
        ax2.plot(self.__draw[0][1:],self.__draw[-2][1:],label='test accuracy')
        ax2.set_title('test accuracy')
        ax2.set_xlabel('n_iteration')
        ax2.legend(loc='upper right')
        
        fig4, ax4 = plt.subplots()
        ax4.plot(self.__draw[0][1:],self.__draw[-4][1:],label='dx')
        ax4.set_title('dx')
        ax4.set_xlabel('n_iteration')
        ax4.legend(loc='upper right')

        fig3, ax3 = plt.subplots()
        for i in range(len(self.__w)):
            ax3.plot(self.__draw[0],self.__draw[i+1],linewidth=1,color=colors[i],label='w{}'.format(i))
        ax3.set_title('evaluation_var')
        ax3.set_xlabel('n_iteration')
        ax3.legend(loc='upper right')

        fig6, ax6 = plt.subplots()
        ax6.plot(self.__draw[0][1:],self.__draw[-1][1:],label='time')
        ax6.set_title('time')
        ax6.set_xlabel('n_iteration')
        ax6.legend(loc='upper right')

    def SGD(self):
        n_satisfy_eps=0
        V=np.zeros(len(self.__w),dtype=np.float32)
        while self.__n<=self.__nmax and n_satisfy_eps<=10  and self.__error>=self.__error_limited:
            #print('1')
            list1=[self.__n]
            self.append_list(list1,[self.__w[i].item() for i in range(len(self.__w))])
            self.append_list(list1,[self.__dx,self.get_error(),self.classification(self.__w),time.time()-self.__time_initial])
            self.draw=list1

            gradient=self.getgradient()
            #mise a jour quantite du mouvement
            V=[self.__momentum*V[i]-self.__alpha*gradient[i].item() for i in range(len(self.__w))] 
            #apres d'avoir obtenu les gradient (dA,dB,dC,dgamma),on va renouveller A,B,C,gamma par le taux d'apprentissage alpha
            w=[self.__w[i].item()+V[i] for i in range(len(self.__w))]
            #renouveller dx pour verifier l'état de convergence 
            self.dx=(w)
            if self.__dx<=self.__eps:
                n_satisfy_eps+=1
            else:
                n_satisfy_eps=0
            self.w=w
            #generer un nouveau point(xs,ys) qui va utilise pour la prochain itération
            self.__index=random.choice([i for i in range(len(x_train))])
            #point aleatoire depart
            self.__xs=[torch.tensor(item, dtype=torch.float32, requires_grad=False) for item in x_train.T.iloc[:,self.__index]] 
            self.__ys=torch.tensor(y_train[self.__index], dtype=torch.float32, requires_grad=False)
            self.n=self.__n+1
            self.time_initial=time.time()-self.__time_initial
            self.classification(self.__w)
            self.get_error()
        self.Plot_curves()
        return 

      

data = pd.read_csv("brca.csv")
data = data.drop(['Unnamed: 0'], axis=1)
data.y = [1 if each == "M" else 0 for each in data.y]
y = data.y.values
#print(y)
x_data = data.drop(["y"], axis=1)
#print(x_data)
# normalization
x = (x_data - x_data.min())/(x_data.max() - x_data.min()).values
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
x_train['x_b'] = 1
x_test['x_b'] = 1
w=np.ones(x_data.shape[1]+1, dtype=int)
rg_w=[True for i in range(x_data.shape[1]+1)]
            
device="mps" #'mps' pour le gpu d'apple;'cuda'pour gpu nvidia, s'il y a pas de carte graphique---->'cpu'
eps=10e-8
error_limited=0.2
nmax=3000

#SGD
alpha=0.05
momentum=0.8
H = Bra( w,rg_w,device,x_train,y_train,x_test,y_test,eps,error_limited,nmax,alpha,momentum)
#H.getgradient()
#H.get_error()
H.SGD()